In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('org_final_joined_new.csv')

/opt/anaconda3/envs/UBS/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (12,13,17,18,19,20,21,22,23,24,27,28,29,30,31,32,33,34,35,38,41,43,44,45,46,47,48,49,50,52,53,54,55,56,57,58,59,62,63,64,65,66,67,72,73,74,75,76,77,100,101,118) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df.drop(df.iloc[:, 0:43], inplace = True, axis = 1)
df.drop(df.iloc[:, -54:], inplace = True, axis = 1)
df.drop(columns=['partner_domain','investor_domain', 'ipo_stock_exchange_symbol'], inplace = True)

In [4]:
#Filling 0 for missing value for counts
df['partner_investment_count'] = df['partner_investment_count'].fillna(0)
df['investor_investment_count'] = df['investor_investment_count'].fillna(0)

In [5]:
#Fillna for total funds
df['investor_name'] = df['investor_name'].fillna("")
mean = df['investor_total_funding_usd'].mean()
#Selecting 0 or mean to be filled
def fillwhat(df): 
    if df['investor_name']=="" :
        return 0
    else :
        return mean

fill_df = df.copy()
fill_df = fill_df[['investor_total_funding_usd','investor_name']].apply(fillwhat, axis=1)
fill_df = pd.DataFrame(fill_df)
df['investor_total_funding_usd'] = fill_df

In [6]:
#Fillna for acquisition prices
df['acquisitions_acquirer_name'] = df['acquisitions_acquirer_name'].fillna("")
mean = df['acquisitions_price_usd'].mean()
#Selecting 0 or mean to be filled
def fillwhat(df): 
    if df['acquisitions_acquirer_name']=="" :
        return 0
    else :
        return mean

fill_df = df.copy()
fill_df = fill_df[['acquisitions_price_usd','acquisitions_acquirer_name']].apply(fillwhat, axis=1)
fill_df = pd.DataFrame(fill_df)
df['acquisitions_price_usd'] = fill_df

In [7]:
#Fillna for parent_organizations' ranking
mean = df['parent_org_rank'].mean()
#Selecting 0 or mean to be filled
def fillwhat(df): 
    if df['acquisitions_acquired_on']=="" :
        return 0
    else :
        return mean

fill_df = df.copy()
fill_df = fill_df[['parent_org_rank','acquisitions_acquired_on']].apply(fillwhat, axis=1)
fill_df = pd.DataFrame(fill_df)
df['parent_org_rank'] = fill_df

In [8]:
one_hot_pn_tp = pd.get_dummies(df['partner_type'], prefix='partner_type')
one_hot_pn_rl = pd.get_dummies(df['partner_roles'], prefix='partner_roles')
one_hot_pn_cc = pd.get_dummies(df['partner_country_code'], prefix='partner_country')
one_hot_pn_ct = pd.get_dummies(df['partner_city'], prefix='partner_city')

# Drop the encoded columns
df = df.drop(['partner_type','partner_roles','partner_country_code','partner_city'],axis = 1)
# Join the encoded df
df = df.join(one_hot_pn_tp)
df = df.join(one_hot_pn_rl)
df = df.join(one_hot_pn_cc)
df = df.join(one_hot_pn_ct)

In [9]:
#types_series = df['partner_investor_types'].value_counts()
#print(types_series)

In [10]:
df['partner_investor_types'] = df['partner_investor_types'].fillna("")
investment_partner = []
angel = []

#If match append 1 otherwise append 0. Equivalent for one-hot encoding
for i in df['partner_investor_types']:
    if "investment_partner" in i:
        investment_partner.append(1)
    else:
        investment_partner.append(0)
    if "angel" in i:
        angel.append(1)
    else:
        angel.append(0)
        
#Drop the encoded column
df=df.drop(['partner_investor_types'],axis=1)
df['investment_partner'] = investment_partner
df['ange'] = angel

In [11]:
#types_series = df['investor_investor_types'].value_counts().nlargest(30)
#print(types_series)

In [12]:
one_hot_in_tp = pd.get_dummies(df['investor_type'], prefix='investor_type')
one_hot_in_rl = pd.get_dummies(df['investor_roles'], prefix='investor_roles')
one_hot_in_cc = pd.get_dummies(df['investor_country_code'], prefix='investor_country')
one_hot_in_ct = pd.get_dummies(df['investor_city'], prefix='investor_city')

# Drop the encoded columns
df = df.drop(['investor_type','investor_roles','investor_country_code','investor_city'],axis = 1)
# Join the encoded df
df = df.join(one_hot_in_tp)
df = df.join(one_hot_in_rl)
df = df.join(one_hot_in_cc)
df = df.join(one_hot_in_ct)

In [13]:
df['investor_investor_types'] = df['investor_investor_types'].fillna("")
venture_capital = []
micro_vc = []
private_equity_firm = []
corporate_venture_capital = []
entrepreneurship_program = []
government_office =[]
incubator = []
accelerator = []
angel_group = []
angel = []
family_investment_office = []
pension_funds = []
hedge_fund = []
fund_of_funds = []
investment_bank  = []
secondary_purchaser = []
investment_partner = []
venture_debt = []

inv_type_name_arr = ["venture_capital","micro_vc","private_equity_firm","corporate_venture_capital","entrepreneurship_program",\
                "government_office","incubator","accelerator","angel_group","angel","family_investment_office","pension_funds",\
                "hedge_fund", "fund_of_funds", "investment_bank", "secondary_purchaser", "investment_partner", "venture_debt"]
inv_type_arr = [venture_capital,micro_vc,private_equity_firm,corporate_venture_capital,entrepreneurship_program,\
                government_office,incubator,accelerator,angel_group,angel,family_investment_office,pension_funds,\
                hedge_fund, fund_of_funds, investment_bank, secondary_purchaser, investment_partner, venture_debt]

#Loop through both the df and the array. If match, the sub-array will append 1 otherwise 0 for one-hot encoding
for i in df['investor_investor_types']:
    for j in range (len(inv_type_name_arr)):
        if inv_type_name_arr[j] in i:
            inv_type_arr[j].append(1)
        else:
            inv_type_arr[j].append(0)
            
#Drop the encoded column
df=df.drop(['investor_investor_types'],axis=1)
for k in range (len(inv_type_name_arr)):
    df[inv_type_name_arr[k]] = inv_type_arr[k]

In [14]:
one_hot_ip_cc = pd.get_dummies(df['ipo_country_code'], prefix='ipo_country')
one_hot_ip_ct = pd.get_dummies(df['ipo_city'], prefix='ipo_city')
one_hot_ac_cc = pd.get_dummies(df['acquisitions_acquirer_country_code'], prefix='acquirer_country')
one_hot_ac_ct = pd.get_dummies(df['acquisitions_acquirer_city'], prefix='acquirer_city')

# Drop the encoded columns
df = df.drop(['ipo_country_code','ipo_city','acquisitions_acquirer_country_code','acquisitions_acquirer_city'],axis = 1)
# Join the encoded df
df = df.join(one_hot_ip_cc)
df = df.join(one_hot_ip_ct)
df = df.join(one_hot_ac_cc)
df = df.join(one_hot_ac_ct)

In [15]:
#Fill mean values for datetime. "Investor_founded_on" and "acquisitions_acquired_on"

#Firstly, convert the dates into pandas-compatible datetime value
df['investor_founded_on'] = pd.to_datetime(df['investor_founded_on'], infer_datetime_format=True)

#Next, change datatime to their corresponding integer value to compute the mean
df['investor_founded_on'] = df['investor_founded_on'].apply(lambda x: x.value)
mean = df['investor_founded_on'].mean()

def fillwhat(df): 
    if df['investor_name'] == "" :
        return ""
    else :
        return mean

fill_df = df.copy()
fill_df = fill_df[['investor_founded_on','investor_name']].apply(fillwhat, axis=1)
fill_df = pd.DataFrame(fill_df)
df['investor_founded_on'] = fill_df

#Finally, change the integer back to datetime (optional)
df['investor_founded_on'] = df['investor_founded_on'].apply(pd.Timestamp)

In [16]:
#Again, for acquisitions date

#Firstly, convert the dates into pandas-compatible datetime value
df['acquisitions_acquired_on'] = pd.to_datetime(df['acquisitions_acquired_on'], infer_datetime_format=True)

#Next, change datatime to their corresponding integer value to compute the mean
df['acquisitions_acquired_on'] = df['acquisitions_acquired_on'].apply(lambda x: x.value)
mean = df['acquisitions_acquired_on'].mean()

def fillwhat(df): 
    if df['acquisitions_acquirer_name'] == "" :
        return ""
    else :
        return mean

fill_df = df.copy()
fill_df = fill_df[['acquisitions_acquired_on','acquisitions_acquirer_name']].apply(fillwhat, axis=1)
fill_df = pd.DataFrame(fill_df)
df['acquisitions_acquired_on'] = fill_df

#Finally, change the integer back to datetime (optional)
df['acquisitions_acquired_on'] = df['acquisitions_acquired_on'].apply(pd.Timestamp)

In [18]:
"""
1.IPOs related data are not processed yet
"""
"""
2.Dates are now filled with NaT for null items. If the item is not null (e.g. name or uuid are present),
it is filled with mean value of dates instead.
"""

'\n2.Dates are now filled with NaT for null items. If the item is not null (e.g. name or uuid are present),\nit is filled with mean value of dates instead.\n'

In [19]:
"""
3. Should we do this?
"""
#fill the rest unused but kept NaN data into empty string ""
#df['partner_name'] = df['partner_name'].fillna("")
#df['partner_region'] = df['partner_region'].fillna("")
#df['investor_region'] = df['investor_region'].fillna("")
#df['acquisitions_acquirer_uuid'] = df['acquisitions_acquirer_uuid'].fillna("")
#df['acquisitions_acquirer_region'] = df['acquisitions_acquirer_region'].fillna("")

'\n3. Should we do this?\n'

In [20]:
#See the column headers
for i in range (1,15):
    print(df.iloc[:,100*(i-1):100*i].info())
print(df.iloc[:,100*14:].info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1018183 entries, 0 to 1018182
Data columns (total 100 columns):
 #   Column                        Non-Null Count    Dtype         
---  ------                        --------------    -----         
 0   partner_name                  626902 non-null   object        
 1   partner_region                575476 non-null   object        
 2   partner_investment_count      1018183 non-null  float64       
 3   investor_name                 1018183 non-null  object        
 4   investor_region               625614 non-null   object        
 5   investor_investment_count     1018183 non-null  float64       
 6   investor_total_funding_usd    1018183 non-null  float64       
 7   investor_founded_on           626902 non-null   datetime64[ns]
 8   ipo_region                    9299 non-null     object        
 9   ipo_went_public_on            9299 non-null     object        
 10  ipo_share_price_usd           7174 non-null     float64       
 1